# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/patil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/patil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0  92995      0 --:--:-- --:--:-- --:--:-- 92839


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   182k      0 --:--:-- --:--:-- --:--:--  183k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/22 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 12, relationships: 44)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 12, relationships: 44)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [13]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.   

**1. SingleHopSpecificQuerySynthesizer**   

**Purpose:** Generates straightforward, fact-based questions that require retrieving information from one specific section of a document.  
**How it works:** Uses a single node (e.g., a headline or keyphrase) from a knowledge graph built from the input documents to create precise questions.    
  
**2. MultiHopSpecificQuerySynthesizer**  

**Purpose:** Creates questions that require retrieving multiple specific facts from different sections or documents.  
**How it works:** Combines overlapping nodes (e.g., related keyphrases) to form questions demanding cross-referencing.  

**3. MultiHopAbstractQuerySynthesizer** 

**Purpose:** Generates broad, interpretive questions that demand synthesizing ideas  from a theme or summary of content, rather than just retrieving a single fact.  
**How it works:** Leverages general themes or summaries across nodes to produce open-ended queries needing higher-level understanding or explaination.  

Finally, we can use our `TestSetGenerator` to generate our testset!

In [14]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What role does Microsoft play in the developme...,[The ethics of this space remain diabolically ...,Microsoft Research is mentioned as one of the ...,single_hop_specifc_query_synthesizer
1,What challenges do LLMs present according to t...,"[and software engineer, LLMs are infuriating. ...","According to the context, LLMs are infuriating...",single_hop_specifc_query_synthesizer
2,Who is Simon Willison in the context of AI dev...,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison is mentioned in the context of ...,single_hop_specifc_query_synthesizer
3,How does the context describe the role and sig...,[the document includes some of the clearest ex...,The provided context does not explicitly menti...,single_hop_specifc_query_synthesizer
4,How has Google contributed to advancements in ...,[The environmental impact got better The envir...,Google's contribution in 2024 includes the rel...,single_hop_specifc_query_synthesizer
5,Based on the advancements in large language mo...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, the field of large language models sa...",multi_hop_abstract_query_synthesizer
6,How do the technical challenges like API integ...,"[<1-hop>\n\nbecoming increasingly common, and ...",The context highlights that while LLMs have be...,multi_hop_abstract_query_synthesizer
7,How do the capabilities and limitations of lar...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,Simon Willison’s weblog emphasizes that LLMs a...,multi_hop_abstract_query_synthesizer
8,"How does the development of Claude models, esp...",[<1-hop>\n\nThe environmental impact got bette...,"The development of Claude models, including Cl...",multi_hop_specific_query_synthesizer
9,How do Google’s Gemini models compare to OpenA...,[<1-hop>\n\nGemini apps can speak in a similar...,Google’s Gemini models have introduced feature...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [16]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Google's role in developing large lang...,[The ethics of this space remain diabolically ...,The context mentions that better-than-GPT-3 cl...,single_hop_specifc_query_synthesizer
1,Whaat is the significnce of 2023 in the devlop...,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
2,What is OpenAI?,[the document includes some of the clearest ex...,The context mentions 'openai' as a tag word in...,single_hop_specifc_query_synthesizer
3,What is the Gemiini in the context of AI model...,[The year of slop Synthetic training data work...,The provided context does not mention 'Gemiini...,single_hop_specifc_query_synthesizer
4,How do recent AI research publications highlig...,[<1-hop>\n\nthe document includes some of the ...,The context shows that the document includes d...,multi_hop_abstract_query_synthesizer
5,How do large language models (LLMs) use genera...,[<1-hop>\n\nThe ethics of this space remain di...,Large language models (LLMs) leverage generati...,multi_hop_abstract_query_synthesizer
6,How do multimodal vision and voise and live ca...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, multimodal vision, which includes aud...",multi_hop_abstract_query_synthesizer
7,how important is training data quality and qua...,[<1-hop>\n\nThe ethics of this space remain di...,The context explains that the most important f...,multi_hop_abstract_query_synthesizer
8,Wha china is doin with the LLMs and how they t...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"Based on the context, it is indicated that the...",multi_hop_specific_query_synthesizer
9,How does Google’s development of Gemini’s real...,[<1-hop>\n\nalso pre-announced voice mode for ...,Google has made significant strides in AI by r...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [18]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years new!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years new!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [19]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [20]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [23]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague term in AI with no single, clear, or widely understood meaning. Generally, it is converging on the idea of "AI systems that can go away and act on your behalf," often imagined as AI that performs tasks autonomously for you, similar to a travel agent acting on your behalf. Another perspective sees agents as large language models (LLMs) given access to tools which they can use iteratively to solve problems. However, the term lacks a consistent definition, and there are few if any examples of fully functional agents running in production. The concept is still regarded as "coming soon," dependent on solving difficult challenges like gullibility, which may require achieving artificial general intelligence (AGI).\n\nIn summary, agents are AI systems intended to act autonomously on a user\'s behalf, but their definition, capability, and practical realization remain unclear and largely unrealized.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [29]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [30]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [31]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'brief-canvas-100' at:
https://smith.langchain.com/o/958971ef-386b-4c29-9daa-db720d891cdb/datasets/a3acaf00-cf91-43f4-bc27-53ce32a5a23b/compare?selectedSessions=e8fe31ca-025c-46fa-afd7-dbc665a8d9a0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does Meta's Llama model compare to DeepSee...,Based on the provided context:\n\n**Size:** \...,None,"Meta's Llama models, such as Llama 3.1 405B, a...",1,1,0,8.218539,b41b71c2-1170-4ec8-929f-8ae92b762668,945701bc-ada9-4a2a-a636-f35ef56ccf00
1,how GPT-4o different from GPT-4 and how they u...,Based on the provided context:\n\nGPT-4o (the ...,None,GPT-4o is a version that can run external tool...,0,0,0,7.734342,5ecaf9e2-c06f-4d6a-a520-03e937ebc9f7,c51ca26f-267b-441e-b571-aa985125a98b
2,How does Google’s development of Gemini’s real...,"Based on the context, Google’s development of ...",None,Google has made significant strides in AI by r...,1,1,0,4.532586,443ec2e2-c119-445e-9566-a6ea91ff90f2,260d937e-daa3-45a1-ab23-e4170086ee10
3,Wha china is doin with the LLMs and how they t...,Based on the provided context:\n\n- The best c...,None,"Based on the context, it is indicated that the...",0,0,0,4.757758,ff112f7d-51f0-4e08-b5e0-2f806391cb4b,cc2485b4-ff5a-45d6-a452-ad165422f2d4
4,how important is training data quality and qua...,Training data quality and quantity are critica...,None,The context explains that the most important f...,1,1,0,5.818131,df3b2ada-8e97-4f14-95ae-01855ad62b59,33d022df-cf37-448d-8183-cce9d8fa70dd
5,How do multimodal vision and voise and live ca...,"Multimodal vision, voice, and live camera capa...",None,"In 2024, multimodal vision, which includes aud...",1,1,0,4.808281,499e8ea5-4d1f-413c-8048-3e237717b12d,77185363-511c-4f56-aa33-825acb243274
6,How do large language models (LLMs) use genera...,Based on the provided context:\n\nLarge Langua...,None,Large language models (LLMs) leverage generati...,1,1,0,5.920591,3cbacd7e-de9b-4e77-a7c7-c70046c1f2b4,ffdcdf0a-ffef-401f-a0a8-21ab4ca8ff7e
7,How do recent AI research publications highlig...,Recent AI research publications highlight that...,None,The context shows that the document includes d...,1,1,0,2.445740,0d681952-3385-4cbf-95e3-01a29cf690cc,c0ad6cdd-c45b-4262-a5b3-ec9151d990ee
8,What is the Gemiini in the context of AI model...,Gemini refers to a series of large language mo...,None,The provided context does not mention 'Gemiini...,0,0,0,3.081574,38848c4e-7125-420e-952b-bde4ed10b809,378f60c5-f2aa-4f8c-99ce-b570563e7763
9,What is OpenAI?,"Based on the provided context, OpenAI is an or...",None,The context mentions 'openai' as a tag word in...,1,1,0,3.431866,73191b35-2d6d-4a0e-82bb-c31d567c7ca1,9e10661c-9f51-456e-80e8-fb0bfe0e510a


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [32]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [33]:
rag_documents = docs

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

1. Memory size: Larger chunks consume more memory but reduce overhead per chunk. Smaller chunks are more memory-efficient for sparse data access patterns.  
2. Processing efficiency: Larger chunks often allow for better vectorization and cache utilization, improving CPU throughput. However, if chunks become too large to fit in processor caches, performance may degrade.  
3. I/O performance: When reading/writing data, chunk size affects how efficiently your system interacts with storage. Larger chunks reduce the number of I/O operations but increase latency for individual operations.  
4. Parallelization: Chunk size determines the granularity of parallel processing. Smaller chunks enable better load balancing across processors but increase coordination overhead.  
5. Data locality: Properly sized chunks can improve performance by keeping related data together, enhancing cache hits and reducing page faults.  

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?  

Modifying your embedding model can significantly impact application performance in several ways:  

1. Dimensionality tradeoffs: Higher-dimensional embeddings often capture more semantic information but require more storage and computational resources. Lower dimensions are computationally efficient but may lose important nuances.  

2. Model size and inference speed: Larger embedding models generally produce higher quality representations but have slower inference times and higher memory requirements.  

3. Relevance and accuracy: Different embedding models capture different semantic relationships. Switching models can change which items are considered "similar," directly affecting search quality, recommendations, or classification tasks.  

4. Domain specificity: A model trained on general text will perform differently from one fine-tuned on domain-specific data (legal, medical, etc.). Domain-specific models typically outperform general models in their specialized areas.  

5. Vector database performance: Different embedding models produce vectors with varying characteristics that affect indexing and retrieval speed in vector databases.  

6. Context window limitations: Models with different context windows affect how much text can be embedded at once, impacting how you process longer documents.  
 


In [36]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [37]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [38]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [39]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Alright, here’s the lowdown on agents, straight from the freshest context. Agents are kinda like that mysterious “coming soon” tech — people hype them, but nobody’s really nailed a solid, agreed-upon definition yet. The label “agents” gets thrown around all casual-like, but it’s a wildcard term with no clear meaning in the AI world right now.\n\nAt a high level, agents are often imagined in two big flavors:  \n1. AI systems that act on your behalf, like a digital travel agent doing your bidding in the real world.  \n2. Big language models (LLMs) hooked up to tools, running loops to solve problems — more like a Swiss Army knife of automated tasks.\n\nBut here’s the kicker: a huge challenge holding agents back is *gullibility.* They’re like that friend who believes every rumor you throw at them, which makes trusting them to make legit decisions tricky. This gullibility problem is tied up with the dream of AGI (Artificial General Intelligence), and until we get there, these agent dreams 

Finally, we can evaluate the new chain on the same test set!

In [40]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'earnest-religion-42' at:
https://smith.langchain.com/o/958971ef-386b-4c29-9daa-db720d891cdb/datasets/a3acaf00-cf91-43f4-bc27-53ce32a5a23b/compare?selectedSessions=93cd9d87-bdf3-45fa-ab68-a507053818c1




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does Meta's Llama model compare to DeepSee...,"Alright, here’s the lowdown, straight fire sty...",None,"Meta's Llama models, such as Llama 3.1 405B, a...",1,1,1,6.777001,b41b71c2-1170-4ec8-929f-8ae92b762668,f365d4d7-413e-4232-833a-86495130eeb1
1,how GPT-4o different from GPT-4 and how they u...,"Alright, let me break it down for you, cool st...",None,GPT-4o is a version that can run external tool...,1,0,1,5.033104,5ecaf9e2-c06f-4d6a-a520-03e937ebc9f7,3dc002eb-ae30-4e39-aa4d-c73d913fb333
2,How does Google’s development of Gemini’s real...,"Yo, here’s the lowdown straight from the conte...",None,Google has made significant strides in AI by r...,1,1,1,6.820727,443ec2e2-c119-445e-9566-a6ea91ff90f2,6ff98cf0-0d2d-4f49-9ee7-ae2f9bf6c9e3
3,Wha china is doin with the LLMs and how they t...,"Aight, here’s the lowdown on China and their L...",None,"Based on the context, it is indicated that the...",0,0,1,4.341098,ff112f7d-51f0-4e08-b5e0-2f806391cb4b,0e93f7cf-bdda-4b65-b240-61719ae2f283
4,how important is training data quality and qua...,"Yo, check it—training data quality *and* quant...",None,The context explains that the most important f...,1,1,1,4.506298,df3b2ada-8e97-4f14-95ae-01855ad62b59,57b091e1-0966-4832-ad58-5f5105fb3bcd
5,How do multimodal vision and voise and live ca...,"Yo, check this out—multimodal vision and voice...",None,"In 2024, multimodal vision, which includes aud...",1,1,1,6.999341,499e8ea5-4d1f-413c-8048-3e237717b12d,49f2e96f-cfbb-42e3-8925-1808758d69af
6,How do large language models (LLMs) use genera...,"Alright, here’s the scoop straight from the AI...",None,Large language models (LLMs) leverage generati...,1,0,1,6.053971,3cbacd7e-de9b-4e77-a7c7-c70046c1f2b4,461545ab-a068-44a9-bab6-3b6f108b67e0
7,How do recent AI research publications highlig...,"Yo, here’s the lowdown straight from the conte...",None,The context shows that the document includes d...,1,1,1,4.579155,0d681952-3385-4cbf-95e3-01a29cf690cc,bb19dc18-1856-4c2e-ac4f-c5893edf869b
8,What is the Gemiini in the context of AI model...,"Yo, Gemini in the AI world is Google’s big fle...",None,The provided context does not mention 'Gemiini...,0,0,1,4.486892,38848c4e-7125-420e-952b-bde4ed10b809,605fbe09-2d76-4e07-af8c-db6c32a9f4f0
9,What is OpenAI?,"Yo, OpenAI is the big player in the AI game, d...",None,The context mentions 'openai' as a tag word in...,0,0,1,2.901186,73191b35-2d6d-4a0e-82bb-c31d567c7ca1,d53bcf1f-63ab-42ae-b195-651085330bc4


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### Overall Comparison Table

| Metric       | brief-canvas-100     | earnest-religion-42   | Better System           | Notes                                                   |
|--------------|----------------------|------------------------|--------------------------|----------------------------------------------------------|
| Correctness  | 9 correct, 3 incorrect | 9 correct, 3 incorrect | Tie                     | Both equally accurate.                                  |
| Dopeness     | 0/12                 | 12/12                  | **earnest-religion-42** | Much more engaging output.                              |
| Helpfulness  | 9 helpful, 3 not     | 7 helpful, 5 not       | **brief-canvas-100**    | Gave more useful responses.                             |
| P50 Latency  | **4.78s**            | 4.81s                  | Tie                     | Nearly the same latency for typical case.               |
| P99 Latency  | 8.16s                | **6.98s**              | **earnest-religion-42** | More consistent under high load.                        |

### 🔍 Conclusion
- **Choose `earnest-religion-42`** if  goal is *engaging, stylish answers*.
- **Choose `brief-canvas-100`** if  goal is *more helpful, utility-focused responses*.


![Alt text](images/LangSmith_Experiments_1.png)
